<a href="https://colab.research.google.com/github/Quikks1lver/WayFinder/blob/main/NodeXYExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import files
import xml.etree.ElementTree as et
import json
import time

In [13]:
xml = files.upload()

Saving highways_2.osm to highways_2.osm


In [31]:
# Load in the osm XML file and get the root

mytree = et.parse("highways_2.osm")

myroot = mytree.getroot()

print(myroot.attrib)

{'version': '0.6', 'generator': 'Osmosis 0.48.3'}


In [32]:
# Initialize output dictionary/JS object. Holds node id references : node information (id, x, y, list of ways the node is a part of, etc.)
outputMap: dict = {}

# Map holding node information -- node id : w/lat, long
nodeInfoMap: dict = {}

# Stores temporarily the reference id for each node
nodeId: int = 0

# Stores temporarily long/lat for each node
longX: float = 0.0
latY: float = 0.0

# Map to hold lat/long for nodes, temporarily
nodeGeolocation = {}

# Map to hold way info for the nodes
wayInfoForNodeMap: dict = {}

In [33]:
# create a hash map to store way : node info
numNodes: int = 0

for node in myroot.findall("node"):
  longX = node.attrib["lon"]
  latY = node.attrib["lat"]

  nodeGeolocation = {}
  nodeGeolocation["longX"] = longX
  nodeGeolocation["latY"] = latY

  nodeInfoMap[int(node.attrib["id"])] = nodeGeolocation

  numNodes += 1

In [40]:
# print(nodeInfoMap)
print(numNodes)

4425


In [35]:
count: int = 0
DEBUGGING: bool = False
TEST_X_NODES: bool = False
successfullyFoundStreet: bool = True

# for timing the code
t0 = time.time()

# loop through all nodes
for way in myroot.findall("way"):
  count += 1

  if count > 5 and TEST_X_NODES:
    break


  try:    
    wayInfoForNodeMap = {}
    
    # populate id for way in wayinfo map
    wayInfoForNodeMap["wayId"] = int(way.attrib["id"])
    
    successfullyFoundStreet = False
    # populate name for way in the wayinfo map
    for childTag in way.findall("tag"):
      if childTag.attrib['k'] == "name":
        wayInfoForNodeMap["streetName"] = childTag.attrib["v"]
        successfullyFoundStreet = True
        break
      else:
        continue
      childCount += 1

    if not successfullyFoundStreet:
      continue

    # draw relationships between ways <=> nodes to add into output map    
    for node in way.findall("nd"):        
        nodeId = int(node.attrib["ref"])

        if int(nodeId) not in outputMap.keys():
          longX = nodeInfoMap.get(nodeId)["longX"]
          latY = nodeInfoMap.get(nodeId)["latY"]
          tempWayInfo = []
          tempWayInfo.append(wayInfoForNodeMap["streetName"])
          outputMap[int(nodeId)] = {"id": int(nodeId), "longX": float(longX), "latY": float(latY), "waysInfo": tempWayInfo}
        else:
          outputMap[int(nodeId)]["waysInfo"].append(wayInfoForNodeMap["streetName"])

  except:
    continue

t1 = time.time()

print("============================================================")
print(f"Code took {t1 - t0} to run. Ran through {count} nodes")
print("============================================================")

Code took 0.01761937141418457 to run. Ran through 659 nodes


In [36]:
len(outputMap)

4129

In [39]:
if DEBUGGING:
  for val in outputMap.values():
    print(val)

In [38]:
# the json file where the output must be stored 

with open("highways3.json", "w") as outputFile:
  json.dump(outputMap, outputFile)